In [1]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as colors

import pandas as pd
from datetime import datetime
from datetime import timedelta
from glob import glob
import os
##from sys import argv

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER

import matplotlib.tri as tri

#####################
# Load flight track #
#####################
gpsfld = '/home/b/b381492/gpsfiles/' #gps_20220225_RF00.nc'
plane = 'p5'
RF = '13'
RFfile = glob('{}gps_{}*_RF{}.nc'.format(gpsfld, plane, RF))[0]
track = xr.open_dataset(RFfile)
flightdate = RFfile.split('_')[-2].split(plane)[-1]#[2:]
runyear = flightdate[0:4]
runmonth = flightdate[4:6]
runday = flightdate[6:8]

##################
# Load ICON data #
##################
icon_nwp_path = '/work/bb1086/haloac3/icon_nwp/{}-{}-{}*/'.format(runyear, runmonth, runday)
#icon_nwp_path = '/work/bb1086/haloac3/icon_nwp/{}-{}-{}bla*/'.format(runyear, runmonth, '14') 
icon_nwp_files = icon_nwp_path + 'cloud_DOM01_ML_{}*Z.nc'.format(runyear)
icon_nwp_file = glob(icon_nwp_files)


#icon_les_path = '/work/bb1086/haloac3/icon_les/halo_{}{}{}_f01/'.format(runyear, runmonth, runday)
#icon_les_files = icon_les_path + 'cloud_DOM01_ML_{}*Z.nc'.format(runyear)
#icon_les_file = glob(icon_les_files)

def preprocess(ds): # preprocess dataset
    years = np.floor(ds.time.values*1.0e-4).astype(int)
    months = np.floor((ds.time.values-years*1.0e4)*1.0e-2).astype(int)
    days = np.floor((ds.time.values-years*1.0e4-months*1.0e2)).astype(int)
    seconds = np.round((ds.time.values % 1)*24*3600).astype(int)
    vals = (years-1970, months-1, days-1, seconds, 0.0*seconds) # add 0 nanoseconds
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[s]', '<m8[ns]') # to convert dtype to datetime64[ns]
    times = sum(np.asarray(v, dtype=t) for t, v in zip(types, vals) if v is not None)
    time = xr.IndexVariable(dims='time', data=times, attrs={'standard_name':'time'})
    ds = ds.assign_coords(time=time)
    return ds

from scipy.spatial import KDTree
class model_kdtree(object):
    #KDtree are described on for example on https://towardsdatascience.com/using-scikit-learns-binary-trees-to-efficiently-find-latitude-and-longitude-neighbors-909979bd929b
    def __init__(self, model_ds):
        self.model_ds = model_ds
        self.kdtree = {}
        self.kdtree = KDTree(np.asarray([np.rad2deg(self.model_ds.clat.values.flatten()),
                                         np.rad2deg(self.model_ds.clon.values.flatten())]).T)
    def query(self, flight_ds):
        distance, indices1d = self.kdtree.query(np.asarray([
                    flight_ds.lat.values,
                    flight_ds.lon.values
                ]).T)
        return indices1d

if len(icon_nwp_file):
    drops = ['clon_bnds', 'clat_bnds', 'height_bnds', 'height', 'height_bnds', 'height_2',
             'height_3', 'height_3_bnds', 'height_4',  'tqc_dia', 'tqi_dia', 'tqv_dia',
             'tqr', 'tke']
    icon_nwp_data = xr.open_mfdataset(sorted(icon_nwp_file),
                                      combine='by_coords',
                                      drop_variables=drops,
                                      preprocess=preprocess)
    icon_nwp_data = icon_nwp_data.assign_coords({'ncells':icon_nwp_data.ncells.values})
    nwpKDtree = model_kdtree(icon_nwp_data)
    indices = nwpKDtree.query(track)
    sel_nwp_data = icon_nwp_data.sel(ncells=xr.DataArray(data=indices, coords=[track.time]),
                                time=track.time, method='nearest')
    print("nwp data selected")
    grp_time_data = [pd.to_datetime(t).strftime('%Y%m%d%H%M%S')+'_{:09d}'.format(nc) for t,nc in zip(sel_nwp_data.time.values,
                                                                                                     sel_nwp_data.ncells.values)]
    group_time_cell = xr.DataArray(data=grp_time_data,
                                   dims=['time'], coords={'time':sel_nwp_data.time})
    sel_nwp_data = sel_nwp_data.assign(group_time_cell=group_time_cell)
    data_groups = sel_nwp_data.reset_coords().groupby('group_time_cell')
    nwp_data = data_groups.mean() # they should be all equal
    print("nwp data grouped")
    sel_nwp_data.to_netcdf('sel_nwp_{}_{}_RF{}.nc'.format(flightdate, plane, RF))
    print("done nwp sel")
    nwp_data.to_netcdf('grp_nwp_{}_{}_RF{}.nc'.format(flightdate, plane, RF))
    print("done nwp grp")
   

IndexError: list index out of range

In [ ]:
nwp_data

In [ ]:








if len(icon_les_file):
    drops = ['clon_bnds', 'clat_bnds', 'height_bnds', 'height_2', 'height_3', 'height_4', 'height_4_bnds',
             'theta_v','tqc', 'tqi', 'tqv', 'tqr', 'tqs']
    icon_les_data = xr.open_mfdataset(sorted(icon_les_file),
                                      combine='by_coords',
                                      drop_variables=drops,
                                      preprocess=preprocess)
    icon_les_data = icon_les_data.assign_coords({'ncells':icon_les_data.ncells.values})
    lesKDtree = model_kdtree(icon_les_data)
    indices = lesKDtree.query(track)
    sel_les_data = icon_les_data.sel(ncells=xr.DataArray(data=indices, coords=[track.time]),
                                     time=track.time, method='nearest')
    print("data les selected")
    grp_time_data = [pd.to_datetime(t).strftime('%Y%m%d%H%M%S')+'_{:09d}'.format(nc) for t,nc in zip(sel_les_data.time.values,
                                                                                                     sel_les_data.ncells.values)]
    group_time_cell = xr.DataArray(data=grp_time_data,
                                   dims=['time'], coords={'time':sel_les_data.time})
    sel_les_data = sel_les_data.assign(group_time_cell=group_time_cell)
    data_groups = sel_les_data.reset_coords().groupby('group_time_cell')
    les_data = data_groups.mean() # they should be all equal
    print("les data grouped")
    sel_les_data.to_netcdf('sel_les_{}_RF{}.nc'.format(flightdate, RF))
    print("done les sel")
    les_data.to_netcdf('grp_les_{}_RF{}.nc'.format(flightdate, RF))
    print("done les grp")
